In [1]:
import re
import pandas as pd
import csv
import numpy as np
from nltk.tokenize import RegexpTokenizer
from nltk.stem import SnowballStemmer
import nltk
import spacy
from collections import Counter
from statistics import mean
nlp = spacy.load('de_core_news_sm')
snowStemmer = SnowballStemmer(language='german')
RAW_DATA_PATH = 'data/raw/'

Helper Functions

In [2]:
def flatten(l):
    return [item for sublist in l for item in sublist]

In [3]:
def find_longest_word(word_list):
    longest_word =  max(word_list, key=len)
    return len(longest_word)

In [4]:
def lemmatize_string(input_text):
    doc = nlp(input_text.lower())
    result = ' '.join([x.lemma_ for x in doc])
    doc = nlp(result.title())
    result = ' '.join([x.lemma_ for x in doc]).upper()
    return result

In [5]:
def stemm_List_of_Words(input_text):
    result = [snowStemmer.stem(word).upper() for word in input_text]
    return result

In [6]:
def contains_song(ID, JAHR, MONAT):
    return  len(df.loc[(df['ID'] == ID) & (df['JAHR'] == JAHR) & (df['MONAT'] == MONAT)]) >= 1

In [7]:
def isSeasonal():
    for index, row in df.iterrows():
        if np.isnan(row['SEASONAL?']) or False:
            df.loc[(df['ID'] == row['ID']), 'SEASONAL?'] = contains_song(row['ID'], row['JAHR'] + 1, row['MONAT'])

Data Extraction

In [8]:
df_Lied = pd.read_csv(RAW_DATA_PATH + 'LIED.csv', usecols=['ID','INTERPRET', 'TITEL', 'SPRACHE_DEUTSCH', 'TEXT_TEIL1', 'TEXT_TEIL2', 'TEXT_TEIL3', 'TEXT_TEIL4'])
#print(df_Lied.head())

df_Chart_Position = pd.read_csv(RAW_DATA_PATH + 'CHART_POSITION.csv', usecols=['LIED_ID', 'POSITION', 'DATUM_VON', 'DATUM_BIS'])
#print(df_Chart_Position.head())

#get stopword-list
with open(RAW_DATA_PATH+'Stoppwords.csv', newline='', encoding='UTF-8') as f:
    stopwords_list = list(csv.reader(f))
stopwords_list = [word.upper() for word in flatten(stopwords_list)]

Data Conversion

In [9]:
df_Lied['TEXT'] = df_Lied['TEXT_TEIL1'].fillna('') + df_Lied['TEXT_TEIL2'].fillna('') + df_Lied['TEXT_TEIL3'].fillna('') + df_Lied['TEXT_TEIL4'].fillna('')

df_Chart_Position['DATUM_VON'] = pd.to_datetime(df_Chart_Position['DATUM_VON'])
df_Chart_Position['DATUM_BIS'] = pd.to_datetime(df_Chart_Position['DATUM_BIS'])
df_Chart_Position['DAUER'] = (df_Chart_Position['DATUM_BIS'] - df_Chart_Position['DATUM_VON']).dt.days.astype('int16')
df_Chart_Position['JAHR'] = df_Chart_Position['DATUM_BIS'].dt.year.astype('int16')
df_Chart_Position['MONAT'] =  df_Chart_Position['DATUM_BIS'].dt.month.astype('int16')

Text Preprocessing

In [10]:
df_Lied['processed_TEXT'] = df_Lied['TEXT']

#lemmatization
#df_Lied['processed_TEXT'] = df_Lied.processed_TEXT.apply(lambda text: lemmatize_string(text))

# tokenize
df_Lied['processed_TEXT'] = df_Lied.processed_TEXT.apply(lambda text: nltk.word_tokenize(text))

#Stemming
df_Lied['processed_TEXT'] = df_Lied.processed_TEXT.apply(lambda text: stemm_List_of_Words(text))

#remove stopwords
df_Lied['processed_TEXT'] = df_Lied.processed_TEXT.apply(lambda x: [item for item in x if item not in stopwords_list])

#remove numbers
df_Lied['processed_TEXT'] = df_Lied.processed_TEXT.apply(lambda word_list : [re.sub('\w*\d\w*','', word) for word in word_list])

In [11]:
print(df_Lied['processed_TEXT'][0])

['LASS', 'STRAND', 'WEISST', 'WELCH', 'BRAUCH', 'MAL', 'WIED', 'MEER', 'SAND', 'YEAH', 'BRAUCH', 'MAL', 'WIEDERGRUND', 'BLEIB', 'AI', 'UHR', 'FAELLT', 'BLAU', 'SEH', 'VERSINKT', 'SETZT', 'LANGSAM', 'WASS', 'SPIEGELGLATT', 'BLINKT', 'WOLL', 'SEHNSUCHT', 'MORG', 'HEUT', 'PERFEKT', 'IS', 'SEHNSUCHT', 'MORG', 'HEUT', 'PERFEKT', 'IS', 'STEH', 'NEB', 'ZEIG', 'ECHT', 'LICHT', 'STADT', 'VERBRANNT', 'LASS', 'STRAND', 'KOMM', 'STEIG', 'LEUCHT', 'DIAMANT', 'YEH', 'ALL', 'SALZ', 'HAUT', 'AI', 'SEH', 'GESTOCH', 'SCHARF', 'TUT', 'FAST', 'AUG', 'WEH', 'ENDLICH', 'NOCH', 'NIE', 'GESEH', 'YEHI', 'WOLL', 'SEHNSUCHT', 'MORG', 'HEUT', 'PERFEKT', 'IS', 'SEHNSUCHT', 'MORG', 'HEUT', 'PERFEKT', 'IS', 'STEH', 'NEB', 'ZEIG', 'ECHT', 'LICHT', 'STADT', 'VERBRANNT', 'LASS', 'STRAND', 'LASS', 'STRAND', 'WEISST', 'WELCH', 'LASS', 'STRAND', 'YEAH', 'BRAUCHGRUND', 'BLEIB', 'AI', 'LASS', 'STRAND', 'WEISST', 'WELCH', 'BRAUCH', 'MAL', 'WIED', 'MEER', 'SAND', 'YEAH', 'BRAUCH', 'MAL', 'WIEDERGRUND', 'BLEIB', 'OUH', 'LASS',

In [12]:
print(df_Lied['TEXT'][0])

LASS UNS ZUM STRAND DU WEISST WELCHEN ICH MEIN ICH BRAUCH MAL WIEDER MEER UND SAND  YEAH BRAUCH MAL WIEDERGRUND ZU BLEIBEN  AI DEINE UHR FAELLT INS BLAU WIR SEHEN  WIE DIE ZEIT VERSINKT WIR SETZTEN LANGSAM AUF DAS WASSER SPIEGELGLATT UND BLINKT  WIR WOLLEN DIE SEHNSUCHT NACH MORGEN  WEIL ES HEUTE PERFEKT IS SEHNSUCHT NACH MORGEN  WEIL ES HEUTE PERFEKT IS DU STEHST NEBEN MIR  ZEIGST MIR  DASS ES ECHT IST DIE LICHTER MEINER STADT SIND VERBRANNT LASS UNS ZUM STRAND    KOMM WIR STEIGEN AUS DU LEUCHTEST WIE AUS DIAMANT  YEHE MIT ALL DEM SALZ AUF DER HAUT  AI ICH SEH DICH GESTOCHEN SCHARF ES TUT FAST DEN AUGEN WEH UND ENDLICH BIST DU DA SO HABE ICH DICH NOCH NIE GESEHEN  YEHI  WIR WOLLEN DIE SEHNSUCHT NACH MORGEN  WEIL ES HEUTE PERFEKT IS SEHNSUCHT NACH MORGEN  WEIL ES HEUTE PERFEKT IS DU STEHST NEBEN MIR  ZEIGST MIR  DASS ES ECHT IST DIE LICHTER MEINER STADT SIND VERBRANNT LASS UNS ZUM STRAND   LASS UNS ZUM STRAND DU WEISST WELCHEN ICH MEIN LASS UNS ZUM STRAND  YEAH ICH BRAUCHGRUND ZU BLEIB

Data Selection

In [13]:
df_Lied.drop(['TEXT_TEIL1','TEXT_TEIL2', 'TEXT_TEIL3', 'TEXT_TEIL4', 'SPRACHE_DEUTSCH'], axis=1, inplace=True)
df_Date = df_Chart_Position[['LIED_ID', 'DATUM_VON', 'DATUM_BIS', 'DAUER','JAHR', 'MONAT']]

#not used anymore:
#df_Chart_Position = df_Chart_Position.groupby('LIED_ID').agg({'POSITION':'mean','DAUER':'sum'}).reset_index()
#df_Chart_Position['POSITION'] = df_Chart_Position.POSITION.apply(lambda pos: round(pos))
#----

df_Lied.sort_values(by='ID', inplace=True)
df_Chart_Position.sort_values(by='LIED_ID', inplace=True)
df = pd.concat([df_Lied, df_Chart_Position], axis='columns')
df.drop('LIED_ID', axis=1, inplace=True)
print(df.head())

       ID       INTERPRET   TITEL  \
7219  408     Rumpelstilz  Kiosk    
7220  408     Rumpelstilz  Kiosk    
7763  425  Costa Cordalis  Anita    
7762  425  Costa Cordalis  Anita    
7761  425  Costa Cordalis  Anita    

                                                   TEXT  \
7219  ALSO ER SAMMLE FUER EINEN GUTEN ZWECK  SAGT DE...   
7220  ALSO ER SAMMLE FUER EINEN GUTEN ZWECK  SAGT DE...   
7763  JVUIOUGIVTOH ICH FAND SIE IRGENDWO  ALLEIN IN ...   
7762  JVUIOUGIVTOH ICH FAND SIE IRGENDWO  ALLEIN IN ...   
7761  JVUIOUGIVTOH ICH FAND SIE IRGENDWO  ALLEIN IN ...   

                                         processed_TEXT  POSITION  DATUM_VON  \
7219  [ALSO, SAMML, FUER, GUT, ZWECK, SAGT, FRITZ, S...        50 2009-03-06   
7220  [ALSO, SAMML, FUER, GUT, ZWECK, SAGT, FRITZ, S...         4 1984-10-22   
7763  [JVUIOUGIVTOH, FAND, IRGENDWO, ALLEIN, MEXIKO,...        23 2003-08-25   
7762  [JVUIOUGIVTOH, FAND, IRGENDWO, ALLEIN, MEXIKO,...        39 2003-07-28   
7761  [JVUIOUGIVTOH, F

Feature Creation

In [14]:
df['ANZ_UNIQUE_WOERTER'] = list(len(set(word)) for word in df['processed_TEXT'])
df['MAX_WORT_WDH'] = [max(Counter(text).values()) for text in df['processed_TEXT']]

max_word_list = []
for text in df['processed_TEXT']:
    count = dict(Counter(text).items())
    count = {k: v for k, v in sorted(count.items(), key=lambda item: item[1], reverse=True)}
    key_list = [key for key in count.keys()]
    values_list = [key for key in count.values()]
    text_dict = {'WORD': key_list, 'FREQ': values_list}
    max_word = key_list[0]
    max_word_list.append(max_word)
df['WORT_MAX_WDH'] = max_word_list

df['LAENGE_LAENGSTES_WORT'] = list(len(max(set(word), key=len)) for word in df['processed_TEXT'])

In [15]:
df

,ID,INTERPRET,TITEL,TEXT,processed_TEXT,POSITION,DATUM_VON,DATUM_BIS,DAUER,JAHR,MONAT,ANZ_UNIQUE_WOERTER,MAX_WORT_WDH,WORT_MAX_WDH,LAENGE_LAENGSTES_WORT,RANK_SCORE
7219,408,Rumpelstilz,Kiosk,ALSO ER SAMMLE FUER EINEN GUTEN ZWECK SAGT DE...,"[ALSO, SAMML, FUER, GUT, ZWECK, SAGT, FRITZ, S...",50,2009-03-06,2009-03-12,6,2009,3,83,6,KIOSK,13,1
7220,408,Rumpelstilz,Kiosk,ALSO ER SAMMLE FUER EINEN GUTEN ZWECK SAGT DE...,"[ALSO, SAMML, FUER, GUT, ZWECK, SAGT, FRITZ, S...",4,1984-10-22,1984-10-28,6,1984,10,83,6,KIOSK,13,47
7763,425,Costa Cordalis,Anita,JVUIOUGIVTOH ICH FAND SIE IRGENDWO ALLEIN IN ...,"[JVUIOUGIVTOH, FAND, IRGENDWO, ALLEIN, MEXIKO,...",23,2003-08-25,2003-08-31,6,2003,8,73,16,ANITA,12,28
7762,425,Costa Cordalis,Anita,JVUIOUGIVTOH ICH FAND SIE IRGENDWO ALLEIN IN ...,"[JVUIOUGIVTOH, FAND, IRGENDWO, ALLEIN, MEXIKO,...",39,2003-07-28,2003-08-03,6,2003,8,73,16,ANITA,12,12
7761,425,Costa Cordalis,Anita,JVUIOUGIVTOH ICH FAND SIE IRGENDWO ALLEIN IN ...,"[JVUIOUGIVTOH, FAND, IRGENDWO, ALLEIN, MEXIKO,...",27,2003-07-28,2003-08-03,6,2003,8,73,16,ANITA,12,24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22129,2303823,AnnenMayKantereit,3 Tage am Meer,ICH WEISS WAS ICH KANN TROTZDEM AB UND AN KOM...,"[WEISS, KANN, TROTZD, KOMM, EN, ZWEIFEL, DARAN...",21,1980-11-24,1980-11-30,6,1980,11,45,11,DREI,11,30
22014,2304169,Farid Bang & B-Case,Baller,BALLER LA LA LA ICH ZIEH DIE WAFFE UND ICH...,"[BALL, LA, LA, LA, ZIEH, WAFF, BALL, LA, LA, L...",42,2020-04-10,2020-04-16,6,2020,4,149,42,LA,14,9
22651,2304914,Yung Hurn,Alleine,BABY SAG MIR WAS DU HEUTE MACHST HAST DU ZEI...,"[BABY, SAG, HEUT, MACH, JA, BABY, SAG, HEUT, M...",20,2019-10-18,2019-10-24,6,2019,10,58,28,JA,7,31
21906,2304939,Azet & Zuna,Ohh oh,OH OH AH OH ULTRA PLUS BABY BIN MEIN ...,"[OH, OH, AH, OH, ULTRA, BABY, EIG, CHEF, SITZ,...",5,2019-07-26,2019-08-01,6,2019,8,109,67,OH,13,46


RANK_SCORES

In [35]:
MAX_RANK = 50
df['RANK_SCORE'] = MAX_RANK - df['POSITION'] + 1
df['MAX_RANK_SCORE'] = [max(df.loc[df.ID == id_, 'RANK_SCORE']) for id_ in df['ID']]
df['MEAN_RANK_SCORE'] = [round(mean(df.loc[df.ID == id_, 'RANK_SCORE'])) for id_ in df['ID']]

Percentage of Stopwords

In [16]:
df['NUMBER_OF_STOPWORDS'] = df.TEXT.str.split().apply(lambda x: len(set(x) & set(stopwords_list)))
df['STOPWORD_PERCENTAGE'] = df.NUMBER_OF_STOPWORDS.apply(lambda row: round(row/len(df['TEXT']), ndigits=5))

In [17]:
df.drop('NUMBER_OF_STOPWORDS', axis=1, inplace=True)

In [18]:
df.reset_index(inplace=True)
df.drop('index',axis=1, inplace=True)
print(df.head())

    ID       INTERPRET   TITEL  \
0  408     Rumpelstilz  Kiosk    
1  408     Rumpelstilz  Kiosk    
2  425  Costa Cordalis  Anita    
3  425  Costa Cordalis  Anita    
4  425  Costa Cordalis  Anita    

                                                TEXT  \
0  ALSO ER SAMMLE FUER EINEN GUTEN ZWECK  SAGT DE...   
1  ALSO ER SAMMLE FUER EINEN GUTEN ZWECK  SAGT DE...   
2  JVUIOUGIVTOH ICH FAND SIE IRGENDWO  ALLEIN IN ...   
3  JVUIOUGIVTOH ICH FAND SIE IRGENDWO  ALLEIN IN ...   
4  JVUIOUGIVTOH ICH FAND SIE IRGENDWO  ALLEIN IN ...   

                                      processed_TEXT  POSITION  DATUM_VON  \
0  [ALSO, SAMML, FUER, GUT, ZWECK, SAGT, FRITZ, S...        50 2009-03-06   
1  [ALSO, SAMML, FUER, GUT, ZWECK, SAGT, FRITZ, S...         4 1984-10-22   
2  [JVUIOUGIVTOH, FAND, IRGENDWO, ALLEIN, MEXIKO,...        23 2003-08-25   
3  [JVUIOUGIVTOH, FAND, IRGENDWO, ALLEIN, MEXIKO,...        39 2003-07-28   
4  [JVUIOUGIVTOH, FAND, IRGENDWO, ALLEIN, MEXIKO,...        27 2003-07-28

Title Analysis

In [19]:
#lemmatize title
#df['processed_TITLE'] = df.TITEL.apply(lambda titel: ' '.join([x.lemma_ for x in nlp(titel)]))
#print(df['processed_TITLE'].head())

#tokenize title
tokenizer = RegexpTokenizer(r'\w+')
df['processed_TITLE'] = df.TITEL.apply(lambda titel: [word for word in tokenizer.tokenize(titel)])

#Stemming
df['processed_TITLE'] = df.processed_TITLE.apply(lambda titel: stemm_List_of_Words(titel))

#remove stopwords
df['processed_TITLE'] = df.processed_TITLE.apply(lambda titel: [word for word in titel if word.upper() not in stopwords_list])

#remove numbers
df['processed_TITLE'] = df.processed_TITLE.apply(lambda word_list : [re.sub('\w*\d\w*','NUMBER', word) for word in word_list])

In [20]:
df['LENGTH_TITLE'] = df.TITEL.apply(lambda titel: len(titel))

In [21]:
print(df.head())

    ID       INTERPRET   TITEL  \
0  408     Rumpelstilz  Kiosk    
1  408     Rumpelstilz  Kiosk    
2  425  Costa Cordalis  Anita    
3  425  Costa Cordalis  Anita    
4  425  Costa Cordalis  Anita    

                                                TEXT  \
0  ALSO ER SAMMLE FUER EINEN GUTEN ZWECK  SAGT DE...   
1  ALSO ER SAMMLE FUER EINEN GUTEN ZWECK  SAGT DE...   
2  JVUIOUGIVTOH ICH FAND SIE IRGENDWO  ALLEIN IN ...   
3  JVUIOUGIVTOH ICH FAND SIE IRGENDWO  ALLEIN IN ...   
4  JVUIOUGIVTOH ICH FAND SIE IRGENDWO  ALLEIN IN ...   

                                      processed_TEXT  POSITION  DATUM_VON  \
0  [ALSO, SAMML, FUER, GUT, ZWECK, SAGT, FRITZ, S...        50 2009-03-06   
1  [ALSO, SAMML, FUER, GUT, ZWECK, SAGT, FRITZ, S...         4 1984-10-22   
2  [JVUIOUGIVTOH, FAND, IRGENDWO, ALLEIN, MEXIKO,...        23 2003-08-25   
3  [JVUIOUGIVTOH, FAND, IRGENDWO, ALLEIN, MEXIKO,...        39 2003-07-28   
4  [JVUIOUGIVTOH, FAND, IRGENDWO, ALLEIN, MEXIKO,...        27 2003-07-28

In [22]:
title_list = [title for title in df['processed_TITLE']]
count = dict(Counter(flatten(title_list)).items())
count = {k: v for k, v in sorted(count.items(), key=lambda item: item[1], reverse=True)}
key_list = [key for key in count.keys()]
values_list = [key for key in count.values()]
title_dict = {'WORD': key_list, 'FREQ': values_list}
df_Titel = pd.DataFrame(title_dict)
print(df_Titel.head())

     WORD  FREQ
0  NUMBER  1438
1    LIEB   988
2     ALL   452
3   NACHT   417
4     NUR   375


Seasonal determination

In [23]:
df['SEASONAL?'] = np.nan
isSeasonal()
df['SEASONAL?']

0        False
1        False
2        False
3        False
4        False
         ...  
22757    False
22758    False
22759    False
22760    False
22761    False
Name: SEASONAL?, Length: 22762, dtype: object

MULTILINGUAL DETECTION

In [26]:
df['MULTILINGAL?'] = np.nan

Reshape df

In [37]:
new_cols = ['ID', 'INTERPRET', 'TITEL', 'processed_TITLE', 'TEXT', 'processed_TEXT', 'DATUM_VON', 'DATUM_BIS', 'JAHR', 'MONAT', 'DAUER','ANZ_UNIQUE_WOERTER', 'MAX_WORT_WDH', 'WORT_MAX_WDH','LAENGE_LAENGSTES_WORT', 'STOPWORD_PERCENTAGE', 'LENGTH_TITLE', 'SEASONAL?', 'MULTILINGAL?', 'POSITION', 'RANK_SCORE', 'MAX_RANK_SCORE', 'MEAN_RANK_SCORE']
df=df[new_cols]
df=df.reindex(columns=new_cols)
print(np.shape(df))

(22762, 23)


In [28]:
df

,ID,INTERPRET,TITEL,processed_TITLE,TEXT,processed_TEXT,DATUM_VON,DATUM_BIS,JAHR,MONAT,...,ANZ_UNIQUE_WOERTER,MAX_WORT_WDH,WORT_MAX_WDH,LAENGE_LAENGSTES_WORT,STOPWORD_PERCENTAGE,LENGTH_TITLE,SEASONAL?,MULTILINGAL?,POSITION,RANK_SCORE
0,408,Rumpelstilz,Kiosk,[KIOSK],ALSO ER SAMMLE FUER EINEN GUTEN ZWECK SAGT DE...,"[ALSO, SAMML, FUER, GUT, ZWECK, SAGT, FRITZ, S...",2009-03-06,2009-03-12,2009,3,...,83,6,KIOSK,13,0.00211,6,False,NaN,50,1
1,408,Rumpelstilz,Kiosk,[KIOSK],ALSO ER SAMMLE FUER EINEN GUTEN ZWECK SAGT DE...,"[ALSO, SAMML, FUER, GUT, ZWECK, SAGT, FRITZ, S...",1984-10-22,1984-10-28,1984,10,...,83,6,KIOSK,13,0.00211,6,False,NaN,4,47
2,425,Costa Cordalis,Anita,[ANITA],JVUIOUGIVTOH ICH FAND SIE IRGENDWO ALLEIN IN ...,"[JVUIOUGIVTOH, FAND, IRGENDWO, ALLEIN, MEXIKO,...",2003-08-25,2003-08-31,2003,8,...,73,16,ANITA,12,0.00189,6,False,NaN,23,28
3,425,Costa Cordalis,Anita,[ANITA],JVUIOUGIVTOH ICH FAND SIE IRGENDWO ALLEIN IN ...,"[JVUIOUGIVTOH, FAND, IRGENDWO, ALLEIN, MEXIKO,...",2003-07-28,2003-08-03,2003,8,...,73,16,ANITA,12,0.00189,6,False,NaN,39,12
4,425,Costa Cordalis,Anita,[ANITA],JVUIOUGIVTOH ICH FAND SIE IRGENDWO ALLEIN IN ...,"[JVUIOUGIVTOH, FAND, IRGENDWO, ALLEIN, MEXIKO,...",2003-07-28,2003-08-03,2003,8,...,73,16,ANITA,12,0.00189,6,False,NaN,27,24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22757,2303823,AnnenMayKantereit,3 Tage am Meer,"[NUMBER, TAG, MEER]",ICH WEISS WAS ICH KANN TROTZDEM AB UND AN KOM...,"[WEISS, KANN, TROTZD, KOMM, EN, ZWEIFEL, DARAN...",1980-11-24,1980-11-30,1980,11,...,45,11,DREI,11,0.00114,14,False,NaN,21,30
22758,2304169,Farid Bang & B-Case,Baller,[BALL],BALLER LA LA LA ICH ZIEH DIE WAFFE UND ICH...,"[BALL, LA, LA, LA, ZIEH, WAFF, BALL, LA, LA, L...",2020-04-10,2020-04-16,2020,4,...,149,42,LA,14,0.00264,6,False,NaN,42,9
22759,2304914,Yung Hurn,Alleine,[ALLEIN],BABY SAG MIR WAS DU HEUTE MACHST HAST DU ZEI...,"[BABY, SAG, HEUT, MACH, JA, BABY, SAG, HEUT, M...",2019-10-18,2019-10-24,2019,10,...,58,28,JA,7,0.00185,7,False,NaN,20,31
22760,2304939,Azet & Zuna,Ohh oh,"[OHH, OH]",OH OH AH OH ULTRA PLUS BABY BIN MEIN ...,"[OH, OH, AH, OH, ULTRA, BABY, EIG, CHEF, SITZ,...",2019-07-26,2019-08-01,2019,8,...,109,67,OH,13,0.00224,6,False,NaN,5,46


Export

In [38]:
df.to_csv('Data/processed/EDA.csv')
#df_Date.to_csv('Data/processed/DATE.csv')
#df_Titel.to_csv('Data/processed/TITLE-ANALYSIS.csv')